In [2]:
import yfinance as yf
import pandas as pd
import datetime as dt
import pandas_datareader.data as web

import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime as dt
import plotly.graph_objects as go

In [27]:
start = dt.datetime(2024, 1, 1)
end = dt.datetime(2024, 12, 30) #dt.datetime.now()

# Diccionario con nombres de índices y sus símbolos de Yahoo Finance
us_indices = {
    'S&P 500': '^GSPC',
    'Nasdaq': '^IXIC',
    'Dow Jones': '^DJI',
    'Russell 2000': '^RUT'
}

europe_indices = {
    'FTSE 100': '^FTSE',
    'DAX': '^GDAXI',
    'CAC 40': '^FCHI',
    'Euro Stoxx 50': '^STOXX50E',
    'Euronext 100': '^N100'
}

latam_indices = {
    'Bovespa': '^BVSP',
    # 'IPSA (Chile)': '^IPSA',
    'COLCAP (Colombia)': '^COLCAP',
    # 'S&P/BVL General (Peru)': '^SPBLPGPT',
    'MSCI Peru ETF': 'EPU'
}

asia_indices = {
    'Nikkei 225 (Japan)': '^N225',
    'Hang Seng (Hong Kong)': '^HSI',
    'Shanghai Composite (China)': '000001.SS',
    'Kospi (South Korea)': '^KS11'
}

# Función para descargar y normalizar datos
def get_normalized_data(indices, start, end):
    dataframes = []
    for name, ticker in indices.items():
        df = yf.download(ticker, start=start, end=end)[['Close']]
        df.rename(columns={'Close': name}, inplace=True)
        df = df.dropna(subset=[name])
        if not df.empty:
            df[name] = df[name] / df[name].iloc[0] * 100  # Normalizar a base 100
            dataframes.append(df)
    return dataframes

# Descargar y procesar datos para cada región
us_dataframes = get_normalized_data(us_indices, start, end)
europe_dataframes = get_normalized_data(europe_indices, start, end)
latam_dataframes = get_normalized_data(latam_indices, start, end)
asia_dataframes = get_normalized_data(asia_indices, start, end)

# Función para combinar DataFrames
def merge_dataframes(dataframes):
    if dataframes:
        merged_df = dataframes[0]
        for df in dataframes[1:]:
            merged_df = pd.merge(merged_df, df, left_index=True, right_index=True, how='outer')
        return merged_df.fillna(method='ffill').fillna(method='bfill')
    else:
        return pd.DataFrame()

# Combinar DataFrames de cada región
us_merged = merge_dataframes(us_dataframes)
europe_merged = merge_dataframes(europe_dataframes)
latam_merged = merge_dataframes(latam_dataframes)
asia_merged = merge_dataframes(asia_dataframes)

# Función para graficar los índices de una región
def plot_indices(dataframe, title):
    fig = go.Figure()
    for column in dataframe.columns:
        fig.add_trace(go.Scatter(
            x=dataframe.index,
            y=dataframe[column],
            mode='lines',
            name=column
        ))
    fig.update_layout(
        title=title,
        xaxis_title='Fecha',
        yaxis_title='Índice (Base 100)',
        hovermode='x unified'
    )
    fig.show()

# Mostrar gráficos para cada región
plot_indices(us_merged, 'Índices de Estados Unidos (Base 100)')
plot_indices(europe_merged, 'Índices de Europa (Base 100)')
# plot_indices(latam_merged, 'Índices de América Latina (Base 100)')
plot_indices(asia_merged, 'Índices de Asia (Base 100)')


asia_merged, europe_merged, us_merged

C:\Users\HP SUPPORT\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

[*********************100%%**********************]  1 of 1 completed
C:\Users\HP SUPPORT\AppData\Local\Temp\ipykernel_25896\2170451466.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP SUPPORT\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

[*********************100%%**********************]  1 of 1 completed
C:\Users\HP SUPPORT\AppData\Local\Temp\ipykernel_25896\2170451466.py:40: Set

(            Nikkei 225 (Japan)  Hang Seng (Hong Kong)  \
 Date                                                    
 2024-01-02          100.000000             100.000000   
 2024-01-03          100.000000              99.153348   
 2024-01-04          100.000000              99.150788   
 2024-01-05          100.267760              98.491706   
 2024-01-08          100.267760              96.639969   
 ...                        ...                    ...   
 2024-12-23          117.642994             118.432681   
 2024-12-24          117.269024             119.714258   
 2024-12-25          117.550138             119.714258   
 2024-12-26          118.864801             119.714258   
 2024-12-27          121.007001             119.667631   
 
             Shanghai Composite (China)  Kospi (South Korea)  
 Date                                                         
 2024-01-02                  100.000000           100.000000  
 2024-01-03                  100.167817            97.6

In [52]:
import pandas as pd
from openpyxl import load_workbook

# Normalizar índices para eliminar componentes de tiempo
asia.index = asia.index.normalize()
europe.index = europe.index.normalize()
us.index = us.index.normalize()

# Asegurarse de que los índices no tengan nombres
asia.index.name = None
europe.index.name = None
us.index.name = None

# Realizar el merge secuencial
asia_eu = asia.merge(europe, left_index=True, right_index=True, how='inner')
asia_eu_us = asia_eu.merge(us, left_index=True, right_index=True, how='inner')

# Restablecer el índice y renombrarlo como 'time'
asia_eu_us.reset_index(inplace=True)
asia_eu_us.set_index("index", inplace=True)
asia_eu_us.index.name = "time"

# Exportar a una nueva hoja de un archivo de Excel existente
output_file = "merged_data.xlsx"  # Nombre del archivo de salida
sheet_name = "Asia_EU_US_Merged"  # Nombre de la nueva hoja

# # Cargar el archivo existente y agregar una nueva hoja
# with pd.ExcelWriter(output_file, engine='openpyxl', mode='a') as writer:
#     asia_eu_us.to_excel(writer, sheet_name=sheet_name)

# print("Datos exportados exitosamente a la hoja:", sheet_name)
asia_eu_us

,Nikkei 225 (Japan),Hang Seng (Hong Kong),Shanghai Composite (China),Kospi (South Korea),FTSE 100,DAX,CAC 40,Euro Stoxx 50,Euronext 100,S&P 500,Nasdaq,Dow Jones,Russell 2000
time,,,,,,,,,,,,,
2024-01-02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
2024-01-03,100.000000,99.153348,100.167817,97.659009,99.492324,98.622674,98.419835,100.000000,98.987176,99.198369,98.823438,99.244737,97.337038
2024-01-04,100.000000,99.150788,99.732410,96.899029,100.020723,99.093166,98.934650,100.581815,99.617954,98.858279,98.268714,99.271645,97.264007
2024-01-05,100.267760,98.491706,98.882753,96.564176,99.586869,98.955545,98.537087,100.345761,99.465278,99.038763,98.361973,99.339972,96.936604
2024-01-08,100.267760,96.639969,97.477014,96.179878,99.646444,99.684611,98.929476,100.839681,99.555594,100.436658,100.527085,99.915081,98.818062
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-19,116.598297,117.654645,113.764993,91.239821,104.970534,119.085404,96.859725,109.686545,103.290782,123.704202,131.199023,112.268844,110.368638
2024-12-20,116.262805,117.465167,113.698698,90.049474,104.702455,118.577875,96.595609,109.310652,102.994031,125.048758,132.552340,113.589334,111.405508
2024-12-23,117.642994,118.432681,113.131161,91.467556,104.936867,118.363314,96.566925,109.100460,102.921634,125.960022,133.854535,113.766153,111.160567


In [77]:
import pandas as pd
from openpyxl import load_workbook

asia = asia_merged
europe = europe_merged
us = us_merged

asia.index.name = None
europe.index.name = None
us.index.name = None

asia_eu = asia.merge(europe, left_index=True, right_index=True, how='inner')
asia_eu_us_index = asia_eu.merge(us, left_index=True, right_index=True, how='inner')
asia_eu_us_index.columns.name = 'time' 

asia_eu_us_index.to_excel("../Stock Data Yahoo/data/asia_eu_us_index.xlsx",sheet_name="Indices")

# output_file = 'C:/Users/HP SUPPORT/Documents/GitHub/indicators/data/macro_dataset.xlsx'
# # output_file = '../Documents/GitHub/indicators/data/macro_dataset.xlsx'
# sheet_name = "Indices"
# asia_eu_us_index

# with pd.ExcelWriter(output_file, engine='openpyxl', mode='a') as writer:
#     asia_eu_us_index.to_excel(writer, sheet_name=sheet_name)

asia_eu_us_index


C:\Users\HP SUPPORT\AppData\Local\Temp\ipykernel_25896\4120473257.py:16: UserWarning:

Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).



time,Nikkei 225 (Japan),Hang Seng (Hong Kong),Shanghai Composite (China),Kospi (South Korea),FTSE 100,DAX,CAC 40,Euro Stoxx 50,Euronext 100,S&P 500,Nasdaq,Dow Jones,Russell 2000
2024-01-02,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
2024-01-03,100.000000,99.153348,100.167817,97.659009,99.492324,98.622674,98.419835,100.000000,98.987176,99.198369,98.823438,99.244737,97.337038
2024-01-04,100.000000,99.150788,99.732410,96.899029,100.020723,99.093166,98.934650,100.581815,99.617954,98.858279,98.268714,99.271645,97.264007
2024-01-05,100.267760,98.491706,98.882753,96.564176,99.586869,98.955545,98.537087,100.345761,99.465278,99.038763,98.361973,99.339972,96.936604
2024-01-08,100.267760,96.639969,97.477014,96.179878,99.646444,99.684611,98.929476,100.839681,99.555594,100.436658,100.527085,99.915081,98.818062
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-19,116.598297,117.654645,113.764993,91.239821,104.970534,119.085404,96.859725,109.686545,103.290782,123.704202,131.199023,112.268844,110.368638
2024-12-20,116.262805,117.465167,113.698698,90.049474,104.702455,118.577875,96.595609,109.310652,102.994031,125.048758,132.552340,113.589334,111.405508
2024-12-23,117.642994,118.432681,113.131161,91.467556,104.936867,118.363314,96.566925,109.100460,102.921634,125.960022,133.854535,113.766153,111.160567
2024-12-24,117.269024,119.714258,114.558138,91.411747,105.381079,118.363314,96.704627,109.100460,103.135236,127.350969,135.657671,114.800441,112.273949


In [42]:
print(asia.index.equals(europe.index))  # Debe ser True
print(asia.index.equals(us.index))     # Debe ser True


False
False


In [8]:
import yfinance as yf
import pandas as pd
import datetime as dt
import plotly.graph_objects as go

# Fechas de inicio y fin
start = dt.datetime(2024, 1, 1)
end = dt.datetime.now()

# Diccionario con nombres de índices y sus símbolos de Yahoo Finance
# latam_indices = {
#     'Bovespa': '^BVSP',
#     'IPC México': '^MXX',
#     'Merval': '^MERV',
#     'S&P/BVL General': '^SPBLPGPT',
#     'MSCI Peru ETF': 'EPU'
# }
latam_indices = {
    'Bovespa': '^BVSP',  # Brasil
    'IPC México': '^MXX', #Mexico
    'IPSA (Chile)': '^IPSA',  # Chile
    'COLCAP (Colombia)': '^COLCAP',  # Colombia
    'S&P/BVL General (Peru)': '^SPBLPGPT',  # Perú
    'MSCI Peru ETF': 'EPU'  # ETF de Perú como representación del mercado
}


# Descargar y preparar datos de América Latina
latam_dataframes = []
for name, ticker in latam_indices.items():
    df = yf.download(ticker, start=start, end=end)[['Close']]
    df.rename(columns={'Close': name}, inplace=True)
    
    # Eliminar NaN al inicio
    df = df.dropna(subset=[name])
    
    # Verificar si el DataFrame no está vacío antes de la normalización
    if not df.empty:
        df[name] = df[name] / df[name].iloc[0] * 100  # Normalizar a base 100
        latam_dataframes.append(df)

# Verificar que la lista de DataFrames no esté vacía antes del merge
if latam_dataframes:
    latam_merged = latam_dataframes[0]
    for df in latam_dataframes[1:]:
        latam_merged = pd.merge(latam_merged, df, left_index=True, right_index=True, how='outer')


    latam_merged = latam_merged.dropna()


    # Crear gráfico para América Latina
    fig = go.Figure()
    for column in latam_merged.columns:
        fig.add_trace(go.Scatter(
            x=latam_merged.index,
            y=latam_merged[column],
            mode='lines',
            name=column
        ))
    fig.update_layout(
        title='Índices de América Latina (Base 100)',
        xaxis_title='Fecha',
        yaxis_title='Índice (Base 100)',
        hovermode='x unified'
    )
    fig.show()
else:
    print("No hay datos válidos para los índices de América Latina.")


C:\Users\HP SUPPORT\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

[*********************100%%**********************]  1 of 1 completed
C:\Users\HP SUPPORT\AppData\Local\Temp\ipykernel_21068\1624474388.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP SUPPORT\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

[*********************100%%**********************]  1 of 1 completed
C:\Users\HP SUPPORT\AppData\Local\Temp\ipykernel_21068\1624474388.py:32: Set

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Función para obtener los datos de un índice específico
def get_investing_data(index_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Realizamos la solicitud GET al sitio web
    response = requests.get(index_url, headers=headers)
    
    # Si la solicitud fue exitosa
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontramos la tabla de datos históricos (dependiendo de la estructura del HTML)
        table = soup.find('table', {'class': 'genTbl closedTbl historicalTbl'})
        
        # Extraemos las filas de la tabla (cada fila es un día de datos)
        rows = table.find_all('tr')
        
        # Creamos listas para almacenar los datos
        dates = []
        opens = []
        highs = []
        lows = []
        closes = []
        volumes = []
        
        # Iteramos sobre cada fila de datos
        for row in rows[1:]:  # Saltamos la primera fila que es el encabezado
            cols = row.find_all('td')
            if len(cols) > 1:  # Si la fila tiene datos
                dates.append(cols[0].text.strip())
                opens.append(cols[1].text.strip())
                highs.append(cols[2].text.strip())
                lows.append(cols[3].text.strip())
                closes.append(cols[4].text.strip())
                volumes.append(cols[5].text.strip())
        
        # Convertimos las listas en un DataFrame de pandas
        data = {
            'Date': dates,
            'Open': opens,
            'High': highs,
            'Low': lows,
            'Close': closes,
            'Volume': volumes
        }
        
        df = pd.DataFrame(data)
        
        # Convertir las fechas al formato correcto (puedes ajustarlo según la región)
        df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')
        
        return df
    else:
        print("Error al obtener los datos")
        return None

# URL del índice S&P Merval (puedes cambiar la URL para otros índices)
url = 'https://www.investing.com/indices/sp-bvlmerval-historical-data'

# Obtener los datos
data = get_investing_data(url)

# Mostrar los primeros registros
if data is not None:
    print(data.head())

    # Guardar los datos en un archivo CSV
    data.to_csv('sp_merval_data.csv', index=False)


Error al obtener los datos


### LATAM Y CARIBE

IPSA CHILE: https://es.investing.com/indices/ipsa-historical-data

S&P LIMA GENERAL: https://es.investing.com/indices/lima-stock-exchange-general

COLCAP: https://es.investing.com/indices/colcap

S&P/BMV IPC (MXX): https://es.investing.com/indices/ipc

S&P MERVAL: https://es.investing.com/indices/merv
